In [1]:

from fuzzywuzzy import fuzz
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import pandas as pd
from fuzzywuzzy import fuzz
from sklearn.cluster import DBSCAN
import numpy as np

import re

In [2]:
df = pd.read_csv("/Users/mollyrudisill/Voxcroft/cleandata_processed.csv", index_col="Unnamed: 0")
df= df[~df['Name'].str.startswith('HTTPS', na=False)]
df = df[:1000]

/var/folders/dk/95f115l12ms7fs6v815_g44c0000gn/T/ipykernel_67323/3305158804.py:1: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("/Users/mollyrudisill/Voxcroft/cleandata_processed.csv", index_col="Unnamed: 0")


In [3]:
# Calculate similarity scores for all pairs
# Calculate similarity scores for all pairs
pairs = []
for i in range(len(df)):
    for j in range(i + 1, len(df)):
        name1 = df.iloc[i]['Name']
        name2 = df.iloc[j]['Name']
        score = fuzz.ratio(name1, name2)
        pairs.append((name1, name2, score))

pairs_df = pd.DataFrame(pairs, columns=['Name1', 'Name2', 'Score'])

# Determine a threshold for matches (e.g., score >= 85)
threshold = 75
pairs_df['Match'] = pairs_df['Score'] >= threshold

print(pairs_df)

                      Name1               Name2  Score  Match
0               BENNY GANTZ  BENJAMIN NETANYAHU     48  False
1               BENNY GANTZ        YOAV GALLANT     52  False
2               BENNY GANTZ       GADI EISENKOT     33  False
3               BENNY GANTZ    BEZALEL SMOTRICH     30  False
4               BENNY GANTZ       ASSAF SHAPIRA     17  False
...                     ...                 ...    ...    ...
499495  DIPANJAN CHATTERJEE                SIRI     17  False
499496  DIPANJAN CHATTERJEE   ALDERMAN JP SMITH     33  False
499497   SIRI MAKEOVERAPPLE                SIRI     36  False
499498   SIRI MAKEOVERAPPLE   ALDERMAN JP SMITH     29  False
499499                 SIRI   ALDERMAN JP SMITH     19  False

[499500 rows x 4 columns]


In [4]:
def print_matches(pairs_df):
    matches = pairs_df[pairs_df['Match']]
    print(matches)

# Print only the pairs that are matches
true_matches = print_matches(pairs_df)



                     Name1               Name2  Score  Match
147            BENNY GANTZ         BENNY GANTZ    100   True
1146    BENJAMIN NETANYAHU  BENJAMIN NETANYAHU    100   True
1849    BENJAMIN NETANYAHU  BENJAMIN NETANYAHU    100   True
2847          YOAV GALLANT        YOAV GALLANT    100   True
8955            MATHABATHA     STAN MATHABATHA     80   True
...                    ...                 ...    ...    ...
498052          JACOB ZUMA          JACOB ZUMA    100   True
498281         CREDO MUTWA         CREDO MUTWA    100   True
498285         CREDO MUTWA         CREDO MUTWA    100   True
498453   MAMELODI SUNDOWNS   MAMELODI SUNDOWNS    100   True
498600         CREDO MUTWA         CREDO MUTWA    100   True

[1486 rows x 4 columns]


In [5]:
matches = pairs_df[pairs_df['Match']]

matches.head(50)
# Print only the pairs that are matches

,Name1,Name2,Score,Match
147,BENNY GANTZ,BENNY GANTZ,100,True
1146,BENJAMIN NETANYAHU,BENJAMIN NETANYAHU,100,True
1849,BENJAMIN NETANYAHU,BENJAMIN NETANYAHU,100,True
2847,YOAV GALLANT,YOAV GALLANT,100,True
8955,MATHABATHA,STAN MATHABATHA,80,True
9685,MATHABATHA,MATHATHO,78,True
19027,JABULANI KHUMALO,JABULANI KHUMALO,100,True
19230,JABULANI KHUMALO,JABULANI KHUMALO,100,True
19430,JABULANI KHUMALO,JABULANI KHUMALO,100,True
19449,JABULANI KHUMALO,JABULANI KHUMALO,100,True


In [6]:
names = df['Name'].unique()
similarity_matrix = np.zeros((len(names), len(names)))

for i in range(len(names)):
    for j in range(len(names)):
        if i != j:
            similarity_matrix[i, j] = fuzz.ratio(names[i], names[j])

# Convert similarity matrix to distance matrix (1 - similarity)
distance_matrix = 100 - similarity_matrix

# Adjusted DBSCAN clustering
clustering = DBSCAN(eps=30, min_samples=1, metric='precomputed').fit(distance_matrix)

# Check the labels again
print("Clustering Labels and Names:")
for label, name in zip(clustering.labels_, names):
    print(f"Label: {label}, Name: {name}")


Clustering Labels and Names:
Label: -1, Name: BENNY GANTZ
Label: -1, Name: BENJAMIN NETANYAHU
Label: -1, Name: YOAV GALLANT
Label: -1, Name: GADI EISENKOT
Label: -1, Name: BEZALEL SMOTRICH
Label: -1, Name: ASSAF SHAPIRA
Label: -1, Name: INBAL TZACH
Label: -1, Name: TAL SHOHAM
Label: -1, Name: SMOTRICH
Label: 0, Name: MATHABATHA
Label: 0, Name: STAN MATHABATHA
Label: -1, Name: REUBEN MADADZHE
Label: -1, Name: REGGIE MOLOKOMME
Label: -1, Name: DANNY NDLOVU
Label: -1, Name: FRANS MOKWELE
Label: -1, Name: ROSEMARY MOLEPO
Label: -1, Name: JEREMIAH NGOBENI
Label: -1, Name: JIMMY MACHAKA
Label: -1, Name: AXED MK
Label: -1, Name: JABULANI KHUMALO
Label: -1, Name: AJAY GUPTA
Label: 1, Name: LETHABO LEJOY
Label: 2, Name: CHRISTALL KAY
Label: 3, Name: SAULOS CHILIMA
Label: 3, Name: SAULOS KLAUS
Label: -1, Name: AXING
Label: -1, Name: NIGEL SIBANDA/
Label: 4, Name: GRANDI THEUNISSEN
Label: 5, Name: JACO BARKHUIZEN
Label: 5, Name: BARKHUIZEN
Label: 4, Name: THEUNISSEN
Label: -1, Name: NASIPHI MOYA


In [7]:
# Create a dictionary to map clusters to names
name_mappings = {}
for cluster_label in set(clustering.labels_):
    if cluster_label != -1:  # Exclude noise points
        cluster_names = names[clustering.labels_ == cluster_label]
        if cluster_names.size > 0:
            standardized_name = max(cluster_names, key=len)  # Choose the longest name as the standard
            for name in cluster_names:
                name_mappings[name] = standardized_name

# Apply the name mappings to the dataframe
df['Mapped_Name'] = df['Name'].apply(lambda x: name_mappings.get(x, x))

In [8]:
print("Name Mappings:")
for original_name, mapped_name in name_mappings.items():
    print(f"{original_name} -> {mapped_name}")

print("\nDataFrame with Mapped Names:")
print(df[['Name', 'Mapped_Name']])

Name Mappings:
MATHABATHA -> STAN MATHABATHA
STAN MATHABATHA -> STAN MATHABATHA
MATHATHO -> STAN MATHABATHA
LETHABO LEJOY -> LETHABO LEJOY MATHATHO
LETHABO LEJOY MATHATO -> LETHABO LEJOY MATHATHO
LETHABO LEJOY MATHATHO -> LETHABO LEJOY MATHATHO
CHRISTALL KAY -> CHRISTALL KAY
CHRISTALL -> CHRISTALL KAY
SAULOS CHILIMA -> SAULOS KLAUS CHILIMA
SAULOS KLAUS -> SAULOS KLAUS CHILIMA
SAULOS KLAUS CHILIMA -> SAULOS KLAUS CHILIMA
GRANDI THEUNISSEN -> GRANDI THEUNISSEN
THEUNISSEN -> GRANDI THEUNISSEN
JACO BARKHUIZEN -> JACO BARKHUIZEN
BARKHUIZEN -> JACO BARKHUIZEN
YOLANDA FARO -> ROLAND GARROS
ROLAND GARROS -> ROLAND GARROS
GERDA
GERDA STEYN -> GERDA
GERDA STEYN
GERDA STEYN -> GERDA
GERDA STEYN
CYRIL RAMAPHOSA -> CYRIL RAMAPHOSA
RAMAPHOSA -> CYRIL RAMAPHOSA
AMAKHOSI -> CYRIL RAMAPHOSA
NELSON MANDELA -> NELSON MANDELA’S
NELSON MANDELA’S -> NELSON MANDELA’S
CHRIS LIEBENBERG -> CHRIS LIEBENBERG
LIEBENBERG -> CHRIS LIEBENBERG
JADEN HENDRIKSE -> GANIEF HENDRICKS
GANIEF HENDRICKS -> GANIEF HENDRICKS
LA

In [9]:
name_mappings

{'MATHABATHA': 'STAN MATHABATHA',
 'STAN MATHABATHA': 'STAN MATHABATHA',
 'MATHATHO': 'STAN MATHABATHA',
 'LETHABO LEJOY': 'LETHABO LEJOY MATHATHO',
 'LETHABO LEJOY MATHATO': 'LETHABO LEJOY MATHATHO',
 'LETHABO LEJOY MATHATHO': 'LETHABO LEJOY MATHATHO',
 'CHRISTALL KAY': 'CHRISTALL KAY',
 'CHRISTALL': 'CHRISTALL KAY',
 'SAULOS CHILIMA': 'SAULOS KLAUS CHILIMA',
 'SAULOS KLAUS': 'SAULOS KLAUS CHILIMA',
 'SAULOS KLAUS CHILIMA': 'SAULOS KLAUS CHILIMA',
 'GRANDI THEUNISSEN': 'GRANDI THEUNISSEN',
 'THEUNISSEN': 'GRANDI THEUNISSEN',
 'JACO BARKHUIZEN': 'JACO BARKHUIZEN',
 'BARKHUIZEN': 'JACO BARKHUIZEN',
 'YOLANDA FARO': 'ROLAND GARROS',
 'ROLAND GARROS': 'ROLAND GARROS',
 'GERDA\nGERDA STEYN': 'GERDA\nGERDA STEYN',
 'GERDA STEYN': 'GERDA\nGERDA STEYN',
 'CYRIL RAMAPHOSA': 'CYRIL RAMAPHOSA',
 'RAMAPHOSA': 'CYRIL RAMAPHOSA',
 'AMAKHOSI': 'CYRIL RAMAPHOSA',
 'NELSON MANDELA': 'NELSON MANDELA’S',
 'NELSON MANDELA’S': 'NELSON MANDELA’S',
 'CHRIS LIEBENBERG': 'CHRIS LIEBENBERG',
 'LIEBENBERG': 'CH

In [10]:
df.tail(40)

,Article_Date_Published,Article_Body,Name,Identity_Type,Article_Source,Voice,Article_Themes_AI_Model,Article_Subject_Keyword_Identified,Article_Topic_Keyword_Identified,Position,Article,Frequency,Mapped_Name
1030,6/10/24 16:38,Madrid — Three Valencia football fans were sen...,VINICIUS JR,People,businesslive.co.za,Fernando Kallas,"[Primary: Conflict, war and peace|15% |Seconda...",['nan'],['Crime & Terrorism'],"['0', '1']",379.0,2.0,VINICIUS JUNIOR
1031,6/10/24 16:38,The evil @helenzille tried to have Credo Mutwa...,CREDO MUTWA,People,twitter.com,Singela,"[Primary: Conflict, war and peace|59% |Seconda...",['nan'],['nan'],"['0', '2']",380.0,2.0,CREDO MUTWA
1033,6/10/24 16:36,The ANC has received pushback from one of its...,AL JAMA-AH,People,timeslive.co.za,NaN,"[Primary: Conflict, war and peace|60% |Seconda...","['ANC', 'ActionSA', 'DA', 'South Africa']",['Human Rights'],"['0', '1']",381.0,2.0,AL JAMA-AH
1034,6/10/24 16:36,Reading Time: < 1 minute\nNine motorists have ...,CHRIS HANI ROAD,People,sabcnews.com,Otsile Saku,[Primary: Sport|82% |Secondary: Uncategorized|...,"['Johannesburg', 'South Africa']",['State Security'],['0'],382.0,1.0,CHRIS HANI ROAD
1035,6/10/24 16:35,Amazon appears to be struggling to gain tracti...,CHRIS GILMOUR,People,techcentral.co.za,Nkosinathi Ndlovu,[Primary: Environment|53% |Secondary: Uncatego...,['South Africa'],['Armed Conflict'],['0'],383.0,1.0,CHRIS GILMOUR
1036,6/10/24 16:35,Amazon appears to be struggling to gain tracti...,GILMOUR,People,techcentral.co.za,Nkosinathi Ndlovu,[Primary: Environment|53% |Secondary: Uncatego...,['South Africa'],['Armed Conflict'],"['1', '3']",383.0,2.0,CHRIS GILMOUR
1037,6/10/24 16:35,Amazon appears to be struggling to gain tracti...,ARTHUR GOLDSTUCK,People,techcentral.co.za,Nkosinathi Ndlovu,[Primary: Environment|53% |Secondary: Uncatego...,['South Africa'],['Armed Conflict'],['2'],383.0,1.0,ARTHUR GOLDSTUCK
1038,6/10/24 16:33,New York — Jasprit Bumrah appears unaware of ...,BUMRAH,People,businesslive.co.za,Amlan Chakraborty,[Primary: Human interest|52% |Secondary: Uncat...,['nan'],['Human Rights'],"['0', '1']",384.0,2.0,BUMRAH
1039,6/10/24 16:32,Parliament has cancelled accommodation and fli...,RAYMOND ZONDO,People,economy24.co.za,NaN,[Primary: Politics|99% |Secondary: Government ...,"['Cape Town', 'South Africa']",['Civil Unrest & Protest'],['0'],385.0,1.0,RAYMOND ZONDO
1040,6/10/24 16:32,Parliament has cancelled accommodation and fli...,XOLILE GEORGE,People,economy24.co.za,NaN,[Primary: Politics|99% |Secondary: Government ...,"['Cape Town', 'South Africa']",['Civil Unrest & Protest'],['1'],385.0,1.0,XOLILE GEORGE.


In [12]:
df.nunique

<bound method DataFrame.nunique of      Article_Date_Published  \
0              6/11/24 7:34   
1              6/11/24 7:34   
2              6/11/24 7:34   
3              6/11/24 7:34   
4              6/11/24 7:34   
...                     ...   
1068          6/10/24 16:20   
1069          6/10/24 16:20   
1070          6/10/24 16:20   
1071          6/10/24 16:20   
1072          6/10/24 16:16   

                                           Article_Body                 Name  \
0     Reading Time: 3 minutes\nIsrael’s parliament m...          BENNY GANTZ   
1     Reading Time: 3 minutes\nIsrael’s parliament m...   BENJAMIN NETANYAHU   
2     Reading Time: 3 minutes\nIsrael’s parliament m...         YOAV GALLANT   
3     Reading Time: 3 minutes\nIsrael’s parliament m...        GADI EISENKOT   
4     Reading Time: 3 minutes\nIsrael’s parliament m...     BEZALEL SMOTRICH   
...                                                 ...                  ...   
1068  Apple is expected to showc

In [16]:
def similarity_matrix(names):
    size = len(names)
    matrix = np.zeros((size, size))
    for i in range(size):
        for j in range(size):
            if i != j:
                matrix[i][j] = fuzz.token_set_ratio(names[i], names[j])
    return matrix

names = df['Name'].tolist()
sim_matrix = similarity_matrix(names)

# Clustering using DBSCAN
dbscan = DBSCAN(eps=15, min_samples=1, metric='precomputed')
clusters = dbscan.fit_predict(100 - sim_matrix)

# Ensure the length of clusters matches the length of the dataframe
if len(clusters) != len(df):
    raise ValueError(f"Length of clusters ({len(clusters)}) does not match length of dataframe ({len(df)})")

# Add cluster labels to the dataframe
df['Cluster'] = clusters

# Display the result
df

,Article_Date_Published,Article_Body,Name,Identity_Type,Article_Source,Voice,Article_Themes_AI_Model,Article_Subject_Keyword_Identified,Article_Topic_Keyword_Identified,Position,Article,Frequency,Mapped_Name,Cluster
0,6/11/24 7:34,Reading Time: 3 minutes\nIsrael’s parliament m...,BENNY GANTZ,People,sabcnews.com,NaN,"[Primary: Conflict, war and peace|92% |Seconda...",['nan'],"['Armed Conflict', 'Crime & Terrorism', 'State...",['0'],0.0,1.0,BENNY GANTZ,0
1,6/11/24 7:34,Reading Time: 3 minutes\nIsrael’s parliament m...,BENJAMIN NETANYAHU,People,sabcnews.com,NaN,"[Primary: Conflict, war and peace|92% |Seconda...",['nan'],"['Armed Conflict', 'Crime & Terrorism', 'State...",['1'],0.0,1.0,BENJAMIN NETANYAHU,1
2,6/11/24 7:34,Reading Time: 3 minutes\nIsrael’s parliament m...,YOAV GALLANT,People,sabcnews.com,NaN,"[Primary: Conflict, war and peace|92% |Seconda...",['nan'],"['Armed Conflict', 'Crime & Terrorism', 'State...",['2'],0.0,1.0,YOAV GALLANT,2
3,6/11/24 7:34,Reading Time: 3 minutes\nIsrael’s parliament m...,GADI EISENKOT,People,sabcnews.com,NaN,"[Primary: Conflict, war and peace|92% |Seconda...",['nan'],"['Armed Conflict', 'Crime & Terrorism', 'State...",['3'],0.0,1.0,GADI EISENKOT,-1
4,6/11/24 7:34,Reading Time: 3 minutes\nIsrael’s parliament m...,BEZALEL SMOTRICH,People,sabcnews.com,NaN,"[Primary: Conflict, war and peace|92% |Seconda...",['nan'],"['Armed Conflict', 'Crime & Terrorism', 'State...",['4'],0.0,1.0,BEZALEL SMOTRICH,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1068,6/10/24 16:20,Apple is expected to showcase how it is integr...,FORRESTER,People,businesslive.co.za,Aditya Soni,"[Primary: Economy, business and finance|99% |S...",['nan'],"['Armed Conflict', 'Human Rights']",['2'],397.0,1.0,FORRESTER,-1
1069,6/10/24 16:20,Apple is expected to showcase how it is integr...,DIPANJAN CHATTERJEE,People,businesslive.co.za,Aditya Soni,"[Primary: Economy, business and finance|99% |S...",['nan'],"['Armed Conflict', 'Human Rights']",['3'],397.0,1.0,DIPANJAN CHATTERJEE,-1
1070,6/10/24 16:20,Apple is expected to showcase how it is integr...,SIRI MAKEOVERAPPLE,People,businesslive.co.za,Aditya Soni,"[Primary: Economy, business and finance|99% |S...",['nan'],"['Armed Conflict', 'Human Rights']",['4'],397.0,1.0,SIRI MAKEOVERAPPLE,177
1071,6/10/24 16:20,Apple is expected to showcase how it is integr...,SIRI,People,businesslive.co.za,Aditya Soni,"[Primary: Economy, business and finance|99% |S...",['nan'],"['Armed Conflict', 'Human Rights']","['5', '6']",397.0,2.0,SIRI,177


In [17]:
# Define the cluster number you want to view
cluster_number = 60   # Replace with the desired cluster number

# Filter the dataframe to get all names in the specified cluster
cluster_names = df[df['Cluster'] == cluster_number]

# Display the result


cluster_names

,Article_Date_Published,Article_Body,Name,Identity_Type,Article_Source,Voice,Article_Themes_AI_Model,Article_Subject_Keyword_Identified,Article_Topic_Keyword_Identified,Position,Article,Frequency,Mapped_Name,Cluster
176,6/11/24 0:44,Johnathan Paoli\nParliament is steaming ahead ...,XOLILE GEORGE.,People,insidepolitic.co.za,NaN,[Primary: Politics|99% |Secondary: Government ...,"['ANC', 'Cyril Ramaphosa', 'South Africa']",['nan'],['2'],51.0,1.0,XOLILE GEORGE.,60
1040,6/10/24 16:32,Parliament has cancelled accommodation and fli...,XOLILE GEORGE,People,economy24.co.za,NaN,[Primary: Politics|99% |Secondary: Government ...,"['Cape Town', 'South Africa']",['Civil Unrest & Protest'],['1'],385.0,1.0,XOLILE GEORGE.,60


In [18]:
df.to_csv("fuzzy_related_names.csv")